In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
rawHisTransDF=pd.read_csv('Historical-transaction-data.csv')
rawStoreInfDF=pd.read_csv('Store-info.csv')
rawTestDF=pd.read_csv('Testing-data.csv')
rawHisTransDF.dropna(subset=['item_description','invoice_id'], inplace=True)
rawHisTransDF=rawHisTransDF.drop_duplicates()
# convert the date string column to datetime
rawHisTransDF['transaction_date'] = pd.to_datetime(rawHisTransDF['transaction_date'], format='%Y/%m/%d').dt.date
rawHisTransDF.dropna(subset=['item_description','invoice_id'], inplace=True)
rawHisTransDF=rawHisTransDF.drop_duplicates()

In [ ]:
rawHisTransDF

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
rawHisTransDF['item_description'] = le.fit_transform(rawHisTransDF['item_description'])
rawHisTransDF['customer_id'] = le.fit_transform(rawHisTransDF['customer_id'])
rawHisTransDF['shop_id'] = rawHisTransDF['shop_id'].str.replace(r'^SHOP', '').astype(int)
rawStoreInfDF['shop_id'] = rawStoreInfDF['shop_id'].str.replace(r'^SHOP', '').astype(int)
rawStoreInfDF['shop_profile'] = rawStoreInfDF['shop_profile'].replace({'High': 3, 'Moderate': 2, 'Low': 1})
rawStoreInfDF['shop_profile'] = rawStoreInfDF['shop_profile'].fillna(0.0).astype(int)
rawHisTransDF['invoice_id'] = rawHisTransDF['invoice_id'].astype(int)

In [ ]:
rawHisTransDF

In [ ]:
rawHisTransDF['payment']=rawHisTransDF['item_price']*rawHisTransDF['quantity_sold']

In [ ]:
# group the dataframe by the 'group' column and get the size of each group
transactions_by_shop = rawHisTransDF.groupby('shop_id').size().reset_index()

# rename columns of the new dataframe
transactions_by_shop.columns = ['shop_id', 'num_of_transactions']

In [ ]:
# Calculate average price for each item type sold by each shop
avg_prices = rawHisTransDF.groupby(['shop_id', 'item_description'])['item_price'].mean().reset_index()

# Rename 'price' column to 'avg_price'
avg_prices.columns = ['shop_id', 'item_description','times']

# Merge the average prices back into the original DataFrame
# merged_df = pd.merge(merged_df, avg_prices, on=['shop_id', 'item_description'])

In [ ]:
avg_prices

In [ ]:
rawHisTransDF

In [ ]:
result = rawHisTransDF.groupby(['shop_id'])['payment'].sum()

In [ ]:
# merging two dataframes on 'key' column
merged_df = pd.merge(rawStoreInfDF, result, on='shop_id')
merged_df = pd.merge(transactions_by_shop,result,on='shop_id')
merged_df

In [ ]:
rawHisTransDF

In [ ]:
# group by 'group' column and find number of unique values in 'value' column for each group
result = rawHisTransDF.groupby('shop_id')['customer_id'].apply(lambda x: len(set(x))).reset_index()
result

In [ ]:
unique_values = rawHisTransDF['item_description'].unique()
print(unique_values)

In [ ]:
# group the dataframe by the 'group' column and get the size of each group
temp = rawHisTransDF.groupby(['shop_id','item_description']).size().reset_index()

# rename columns of the new dataframe
temp.columns = ['shop_id', 'item_description','times']

In [ ]:
# # Calculate average price for each item type sold by each shop
# avg_prices = rawHisTransDF.groupby(['shop_id', 'item_description'])['item_price'].mean().reset_index()

# # Rename 'price' column to 'avg_price'
# avg_prices.columns = ['shop_id', 'item_description','times']

# # Merge the average prices back into the original DataFrame
# # merged_df = pd.merge(merged_df, avg_prices, on=['shop_id', 'item_description'])

In [ ]:
temp.to_csv('final1.csv')

In [ ]:
# pivot the dataframe to make item_description values as columns
pivoted_df = temp.pivot(index='shop_id', columns='item_description', values='times')

# fill NaN values with 0
pivoted_df.fillna(0, inplace=True)

# resetting the index
pivoted_df = pivoted_df.reset_index()

In [ ]:
pivoted_df

In [ ]:
# pivot the dataframe to make item_description values as columns
pivoted_df_avg = avg_prices.pivot(index='shop_id', columns='item_description', values='times')

# fill NaN values with 0
pivoted_df_avg.fillna(0, inplace=True)

# resetting the index
pivoted_df_avg = pivoted_df_avg.reset_index()
pivoted_df_avg

In [ ]:
# rename every column except the first two columns with an incrementing value
for i, col in enumerate(pivoted_df_avg.columns[2:], start=1):
    pivoted_df_avg = pivoted_df_avg.rename(columns={col: f'Column_{i}'})

In [ ]:
pivoted_df_avg

In [ ]:
pivoted_df_avg.round(2)

In [ ]:
rawStoreInfDF=pd.merge(rawStoreInfDF,pivoted_df_avg,on='shop_id')

In [ ]:
pivoted_df.to_csv('final3.csv')

In [ ]:
rawStoreInfDF=pd.merge(rawStoreInfDF,pivoted_df, on='shop_id')

In [ ]:
rawStoreInfDF=pd.merge(rawStoreInfDF,result, on='shop_id')

In [ ]:
rawStoreInfDF

In [ ]:
merged_df

In [ ]:
# convert all float columns to integer columns
rawStoreInfDF = rawStoreInfDF.applymap(lambda x: int(x) if isinstance(x, float) else x)
rawStoreInfDF

In [ ]:
rawStoreInfDF=pd.merge(rawStoreInfDF,merged_df, on='shop_id')
rawStoreInfDF

In [ ]:
rawStoreInfDF.to_csv('storeinfo.csv', index=False)

In [ ]:
rawStoreInfDF=pd.read_csv('storeinfo.csv')

In [ ]:
rawStoreInfDF

In [ ]:
rawStoreInfDF['payment']=rawStoreInfDF['payment']/rawStoreInfDF['shop_area_sq_ft']
rawStoreInfDF=rawStoreInfDF.drop('shop_area_sq_ft',axis=1)

In [ ]:
# Split the DataFrame into two based on column B
TestDF = rawStoreInfDF[rawStoreInfDF['shop_profile'] == 0].drop(['shop_profile'], axis=1)
TrainDF = rawStoreInfDF[rawStoreInfDF['shop_profile'] != 0]

In [ ]:
# Split Fulldata into training and testing sets
from sklearn.model_selection import train_test_split

column_name = 'shop_id'
unique_categories = TrainDF[column_name].nunique()
categories_in_dataset_1 = int(unique_categories * 0.9)
categories_in_dataset_2 = unique_categories - categories_in_dataset_1
dataset_1_categories = TrainDF[column_name].unique()[:categories_in_dataset_1]
dataset_2_categories = TrainDF[column_name].unique()[categories_in_dataset_1:]

train_data = TrainDF[TrainDF[column_name].isin(dataset_1_categories)]
test_data = TrainDF[TrainDF[column_name].isin(dataset_2_categories)]





#train_data, test_data = train_test_split(TrainDF, test_size=0.01)

In [ ]:
expectedResult=test_data[['shop_id','shop_profile']]

In [ ]:
shop_id_x_test = test_data['shop_id']
shop_id_x_TestDF = TestDF['shop_id']
TestDF=TestDF.drop('shop_id',axis=1)
train_data=train_data.drop('shop_id', axis=1)
test_data=test_data.drop('shop_id',axis=1)

In [ ]:
X_train=train_data.drop(['shop_profile'], axis=1)
y_train=train_data['shop_profile']
X_test= test_data.drop(['shop_profile'], axis=1)
y_test=test_data['shop_profile']
X_testres = X_test.reset_index(drop=True)

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.datasets import load_iris



# # Define the decision tree classifier
# dtc = DecisionTreeClassifier()

# # Define the hyperparameters to tune
# params = {
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [2, 3, 4, 5, 6],
#     'min_samples_split': [2, 3, 4, 5],
#     'min_samples_leaf': [1, 2, 3, 4],
# }

# # Perform grid search to find the best hyperparameters
# grid_search = GridSearchCV(dtc, param_grid=params, cv=5)
# grid_search.fit(X_train, y_train)


# # Print the best hyperparameters and the corresponding score
# print("Best hyperparameters: ", grid_search.best_params_)
# print("Best score: ", grid_search.best_score_)


In [ ]:
# import pandas as pd
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import f1_score

# # Define the logistic regression model
# model_random = RandomForestClassifier(max_depth=25, min_samples_leaf=1, min_samples_split=2, n_estimators=10)

# # Train the model on the training data
# model_random.fit(X_train, y_train)

# # Make predictions on the testing data
# predictions = model_random.predict(X_test)
# predictions_Test_randomforest=model_random.predict(TestDF)

# accu = accuracy_score(y_test, predictions)

# print(accu)
# # print(f1_score(y_test, predictions, average=None))



In [ ]:
# predDf=pd.DataFrame(predictions, columns=['shop_profile'])
# predDf_Test_randomforest=pd.DataFrame(predictions_Test_randomforest, columns=['shop_profile'])
# shop_id_x_test=shop_id_x_test.reset_index()
# shop_id_x_TestDF=shop_id_x_TestDF.reset_index()
# shop_id_x_TestDF=shop_id_x_TestDF.drop('index',axis=1)
# # Concatenate DataFrames
# concatenatedRes_df = pd.concat([shop_id_x_test, predDf], axis=1)
# # concatenatedRes_df = pd.concat([concatenatedRes_df, shop_id_x_test], axis=1)
# # concatenated_df = pd.concat([shop_id_x_test, predDf], ignore_index=True)
# concatenatedRes_df_random = pd.concat([shop_id_x_TestDF, predDf_Test_randomforest], axis=1)
# expectedResult=expectedResult['shop_profile']
# concatenatedRes_df=concatenatedRes_df['shop_profile']
# # Calculate F1 score for each class
# f1_class0 = f1_score(expectedResult, concatenatedRes_df, labels=[1], average='weighted')
# f1_class1 = f1_score(expectedResult, concatenatedRes_df, labels=[2], average='weighted')
# f1_class2 = f1_score(expectedResult, concatenatedRes_df, labels=[3], average='weighted')

# # Calculate average F1 score
# f1_average = (f1_class0 + f1_class1 + f1_class2) / 3

# print(f"F1 score for class 0: {f1_class0:.2f}")
# print(f"F1 score for class 1: {f1_class1:.2f}")
# print(f"F1 score for class 2: {f1_class2:.2f}")
# print(f"Average F1 score: {f1_average:.2f}")

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Define the logistic regression model
model_random = KNeighborsClassifier(n_neighbors=5)

# Train the model on the training data
model_random.fit(X_train, y_train)

# Make predictions on the testing data
predictions = model_random.predict(X_test)
predictions_Test_randomforest=model_random.predict(TestDF)

accu = accuracy_score(y_test, predictions)

print(accu)
# print(f1_score(y_test, predictions, average=None))



In [ ]:
predDf=pd.DataFrame(predictions, columns=['shop_profile'])
predDf_Test_randomforest=pd.DataFrame(predictions_Test_randomforest, columns=['shop_profile'])
shop_id_x_test=shop_id_x_test.reset_index()
shop_id_x_TestDF=shop_id_x_TestDF.reset_index()
shop_id_x_TestDF=shop_id_x_TestDF.drop('index',axis=1)
# Concatenate DataFrames
concatenatedRes_df = pd.concat([shop_id_x_test, predDf], axis=1)
# concatenatedRes_df = pd.concat([concatenatedRes_df, shop_id_x_test], axis=1)
# concatenated_df = pd.concat([shop_id_x_test, predDf], ignore_index=True)
concatenatedRes_df_random = pd.concat([shop_id_x_TestDF, predDf_Test_randomforest], axis=1)
expectedResult=expectedResult['shop_profile']
concatenatedRes_df=concatenatedRes_df['shop_profile']
# Calculate F1 score for each class
f1_class0 = f1_score(expectedResult, concatenatedRes_df, labels=[1], average='weighted')
f1_class1 = f1_score(expectedResult, concatenatedRes_df, labels=[2], average='weighted')
f1_class2 = f1_score(expectedResult, concatenatedRes_df, labels=[3], average='weighted')

# Calculate average F1 score
f1_average = (f1_class0 + f1_class1 + f1_class2) / 3

print(f"F1 score for class 0: {f1_class0:.2f}")
print(f"F1 score for class 1: {f1_class1:.2f}")
print(f"F1 score for class 2: {f1_class2:.2f}")
print(f"Average F1 score: {f1_average:.2f}")

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# import numpy as np

# # Train the first set of models
# model1 = LogisticRegression()
# model1.fit(X_train, y_train)

# model2 = RandomForestClassifier(max_depth=25, min_samples_leaf=1, min_samples_split=2, n_estimators=10)
# model2.fit(X_train, y_train)

# # Make predictions on the testing set using the first set of models
# y_pred_1 = model1.predict(X_test)
# y_pred_2 = model2.predict(X_test)
# y_pred_1_test = model1.predict(TestDF)
# y_pred_2_test = model2.predict(TestDF)

# # Stack the predictions from the first set of models
# X_stack = np.column_stack((y_pred_1, y_pred_2))
# X_stack_test = np.column_stack((y_pred_1_test,y_pred_2_test))

# # Train the final model on the stacked predictions
# final_model = LogisticRegression()
# final_model.fit(X_stack, y_test)

# # Make predictions on the testing set using the final model
# y_pred_int = final_model.predict(X_stack)
# y_pred_int_test = final_model.predict(X_stack_test)

# # Calculate the accuracy of the final predictions
# accuracy = accuracy_score(y_test, y_pred_int)
# print(accuracy)


In [ ]:
y_train = y_train.replace({1: 0, 2: 1, 3: 2})
y_test = y_test.replace({1: 0, 2: 1, 3: 2})

In [ ]:
# # from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# import numpy as np

# # Train the first set of models
# model1 = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
# model1.fit(X_train, y_train)

# model2 = RandomForestClassifier(max_depth=25, min_samples_leaf=1, min_samples_split=2, n_estimators=10)
# model2.fit(X_train, y_train)

# # Make predictions on the testing set using the first set of models
# y_pred_1 = model1.predict(X_test)
# y_pred_2 = model2.predict(X_test)
# y_pred_1_test = model1.predict(TestDF)
# y_pred_2_test = model2.predict(TestDF)

# # Stack the predictions from the first set of models
# X_stack = np.column_stack((y_pred_1, y_pred_2))
# X_stack_test = np.column_stack((y_pred_1_test,y_pred_2_test))

# # Train the final model on the stacked predictions
# final_model = xgb.XGBClassifier(objective='multi:softmax', random_state=42)
# final_model.fit(X_stack, y_test)

# # Make predictions on the testing set using the final model
# y_pred_int = final_model.predict(X_stack)
# y_pred_int_test = final_model.predict(X_stack_test)

# # Calculate the accuracy of the final predictions
# accuracy = accuracy_score(y_test, y_pred_int)
# print(accuracy)


In [ ]:
# # from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.neighbors import KNeighborsClassifier
# import numpy as np

# # Train the first set of models
# model1 = KNeighborsClassifier(n_neighbors=10)
# model1.fit(X_train, y_train)

# model2 = RandomForestClassifier(max_depth=100, min_samples_leaf=5, min_samples_split=2, n_estimators=25)
# model2.fit(X_train, y_train)

# # Make predictions on the testing set using the first set of models
# y_pred_1 = model1.predict(X_test)
# y_pred_2 = model2.predict(X_test)
# y_pred_1_test = model1.predict(TestDF)
# y_pred_2_test = model2.predict(TestDF)

# # Stack the predictions from the first set of models
# X_stack = np.column_stack((y_pred_1, y_pred_2))
# X_stack_test = np.column_stack((y_pred_1_test,y_pred_2_test))

# # Train the final model on the stacked predictions
# final_model = KNeighborsClassifier(n_neighbors=5)
# final_model.fit(X_stack, y_test)

# # Make predictions on the testing set using the final model
# y_pred_int = final_model.predict(X_stack)
# y_pred_int_test = final_model.predict(X_stack_test)

# # Calculate the accuracy of the final predictions
# accuracy = accuracy_score(y_test, y_pred_int)
# print(accuracy)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Create confusion matrix
matrix = confusion_matrix(expectedResult, concatenatedRes_df)

# Visualize confusion matrix
sns.heatmap(matrix, annot=True)
plt.show()

In [ ]:
concatenatedRes_df_random['shop_profile'] = concatenatedRes_df_random['shop_profile'].replace({3: 'High', 2: 'Moderate', 1: 'Low'})

In [ ]:
concatenatedRes_df_random.to_csv('day3_1.csv',index=False)